### Note: We are doing the long non-Pythonic way here, because the Fund system cannot import the textblob libraries.

    from textblob import TextBlob, Word
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    from textblob import TextBlob, Word
    from nltk.stem.snowball import SnowballStemmer
    %matplotlib inline

### Always restart the Kernel prior to running all.
====

## Set up the environment and loop over the files

In [23]:
cd "Q:\DATA\SPRLP\EM\Review of Conditionality\Literature\Program approval SRs\text format"

Q:\DATA\SPRLP\EM\Review of Conditionality\Literature\Program approval SRs\text format


In [2]:
#cd "Q:\DATA\SPRLP\EM\Review of Conditionality\Literature\Program approval SRs\text format\test"

Q:\DATA\SPRLP\EM\Review of Conditionality\Literature\Program approval SRs\text format\test


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

#looping over the directory
directory = os.fsencode('Q:\\DATA\\SPRLP\\EM\\Review of Conditionality\\Literature\Program approval SRs\\text format')

df=pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".txt"):
        with open(filename, encoding = 'utf-8') as f:
            list = [line.rstrip('\n') for line in f]
            temp= pd.DataFrame({'text':list})
            temp['prog']=file
        df=df.append(temp, ignore_index=True)                

C:\PROGRAM FILES\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  # This is added back by InteractiveShellApp.init_path()


## Check the shape of the dataset

In [4]:
#Show head and tail to check what we have.
print(df.shape)
df.head()
#df.tail()

(38442, 2)


,text,prog
0,IMF Country Report No. 15/50,b'Serbia 2015 SBA - Copy.txt'
1,,b'Serbia 2015 SBA - Copy.txt'
2,REPUBLIC OF SERBIA,b'Serbia 2015 SBA - Copy.txt'
3,February 2015,b'Serbia 2015 SBA - Copy.txt'
4,,b'Serbia 2015 SBA - Copy.txt'


## Search for a keyword

In [5]:
#Create a sub dataframe which contains the keywords.
sub_df=df.loc[df.text.str.contains('multiplier')]

In [6]:
print(sub_df.shape)
sub_df.head()

(151, 2)


,text,prog
80,potential and reduce fiscal risks.,b'Serbia 2015 SBA - Copy.txt'
721,(Percent of risk-weighted metric),b'Serbia 2015 SBA - Copy.txt'
844,The outlook is subject to significant downside...,b'Serbia 2015 SBA - Copy.txt'
852,resistance. The realization of these risks wou...,b'Serbia 2015 SBA - Copy.txt'
1249,is needed to strengthen their financial positi...,b'Serbia 2015 SBA - Copy.txt'


In [7]:
print('There are', len(sub_df), 'lines in the text with the word')

There are 151 lines in the text with the word


In [9]:
#Now show 2 lines before and after said word for context.
#In array form: df.iloc[[511,512,513],:].values

context_rows=[]
for ind in sub_df.index:
    context_rows.append(ind-2)
    context_rows.append(ind-1)
    context_rows.append(ind)
    context_rows.append(ind+1)
    context_rows.append(ind+2)

search_res= pd.DataFrame(df.iloc[context_rows,:]).reset_index(drop=False)
search_res.head()

,index,text,prog
0,78,Structural reforms. Broad-based structural ref...,b'Serbia 2015 SBA - Copy.txt'
1,79,"environment and resolve loss-making SOEs, shou...",b'Serbia 2015 SBA - Copy.txt'
2,80,potential and reduce fiscal risks.,b'Serbia 2015 SBA - Copy.txt'
3,81,Program modalities. The proposed program suppo...,b'Serbia 2015 SBA - Copy.txt'
4,82,"policy goals to restore fiscal sustainability,...",b'Serbia 2015 SBA - Copy.txt'


In [20]:
search_res['prog']=search_res['prog'].str.decode('utf-8')

In [21]:
#let's relabel
re_index=[]
for i in range(len(sub_df.index)):
    re_index.append(i)
    re_index.append(i)
    re_index.append(i)
    re_index.append(i)
    re_index.append(i)
    
import numpy as np
search_res['doc#'] = pd.DataFrame(np.column_stack([re_index]), columns=['count'])
search_res.head()

,index,text,prog,doc#
0,78,Structural reforms. Broad-based structural ref...,Serbia 2015 SBA - Copy.txt,0
1,79,"environment and resolve loss-making SOEs, shou...",Serbia 2015 SBA - Copy.txt,0
2,80,potential and reduce fiscal risks.,Serbia 2015 SBA - Copy.txt,0
3,81,Program modalities. The proposed program suppo...,Serbia 2015 SBA - Copy.txt,0
4,82,"policy goals to restore fiscal sustainability,...",Serbia 2015 SBA - Copy.txt,0


In [22]:
#Export results to Excel
writer = pd.ExcelWriter('output.xlsx')
search_res.to_excel(writer, 'Sheet1')
writer.save()